In [1]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [2]:
# read SAS datafile for event 1
# alternative is to use pandas_datareader or yfinance to calculate returns for the events
event_1 = pd.read_sas('D:\Schulich\Study Material\Term 1\Econometrics 6310\Assignment 2\Event Study SAS Files Complete\Event1.sas7bdat')
event_1.head()

,PERMNO,DATE,COMNAM,TICKER,RET,SPRTRN
0,12490.0,2013-11-11,b'INTERNATIONAL BUSINESS MACHS COR',b'IBM',0.016056,0.000723
1,12490.0,2013-11-12,b'INTERNATIONAL BUSINESS MACHS COR',b'IBM',0.001039,-0.002370
2,12490.0,2013-11-13,b'INTERNATIONAL BUSINESS MACHS COR',b'IBM',0.002622,0.008095
3,12490.0,2013-11-14,b'INTERNATIONAL BUSINESS MACHS COR',b'IBM',-0.007300,0.004837
4,12490.0,2013-11-15,b'INTERNATIONAL BUSINESS MACHS COR',b'IBM',0.005378,0.004222


In [3]:
# read SAS datafiles for all 24 events
# n is number of events, m is size of estimation window, o is size of event window
n = 24
m = 50
o = 5

events ={}
for i in range(1,n+1):
    path = 'D:\Schulich\Study Material\Term 1\Econometrics 6310\Assignment 2\Event Study SAS Files Complete\Event'+str(i)+'.sas7bdat'
    event = pd.read_sas(path)
    event = event.drop(columns = ['PERMNO','COMNAM','TICKER'])
    events['event%s' %i] = event

In [4]:
# check random event from the dictionary
events['event19'].head()

,DATE,RET,SPRTRN
0,2018-05-16,0.006192,0.004061
1,2018-05-17,-0.000899,-0.000856
2,2018-05-18,-0.002907,-0.002632
3,2018-05-21,0.009786,0.007387
4,2018-05-22,-0.002749,-0.003136


In [5]:
# run regression to build market model using event 1
x = events['event1'].SPRTRN[:-5]
y = events['event1'].RET[:-5]
x = sm.add_constant(x)
model = sm.OLS(y,x).fit()
print(model.params)

const    -0.000229
SPRTRN    0.892884
dtype: float64


In [6]:
# run regression to buils market model using all 24 events
models={}
for i in range(1,n+1):
    x = events['event%s' %i].SPRTRN[:-5]
    y = events['event%s' %i].RET[:-5]
    x = sm.add_constant(x)
    model = sm.OLS(y,x).fit()
    models['model%s' %i] = model.params

In [7]:
models['model17']

const     0.000867
SPRTRN    0.664941
dtype: float64

In [8]:
# calculating abnormal returns using market model
# size of event window is 5 and size of estimation window is 50
abnormal_returns = {}
for i in range (1,n+1):
    temp = []
    for j in range(1,o+1):
        k = m - 1 + j
        expected_return = models['model%s' %i].const + models['model%s' %i].SPRTRN * events['event%s'%i].SPRTRN[k]
        abnormal_return = events['event%s' %i].RET[k] - expected_return
        temp.append(abnormal_return)
    abnormal_returns['event%s' %i] = temp
abnormal_returns = pd.DataFrame(abnormal_returns)

In [9]:
abnormal_returns

,event1,event2,event3,event4,event5,event6,event7,event8,event9,event10,...,event15,event16,event17,event18,event19,event20,event21,event22,event23,event24
0,0.001958,0.001823,0.001389,-0.000278,0.009716,-0.006565,0.000918,-0.005146,-0.021485,-0.007149,...,-0.003004,-0.007417,0.002560,-0.009815,-0.004220,0.005126,-0.000597,0.001301,-0.003335,0.006759
1,-0.005104,0.015513,0.007432,0.002129,0.001664,0.010108,0.003327,-0.003977,0.010957,0.001876,...,-0.006232,0.006224,0.000382,0.007276,0.008015,-0.033290,0.007520,-0.002945,-0.002356,-0.001086
2,-0.011157,0.006137,-0.000824,0.009264,-0.002636,0.014145,-0.006807,-0.036691,-0.008411,-0.001525,...,0.001089,-0.003720,-0.012685,0.011872,-0.007932,-0.048668,-0.000823,0.008772,-0.005409,-0.014267
3,0.006800,0.004144,-0.002359,0.002527,0.000881,0.006747,-0.000736,0.011532,-0.003235,0.006174,...,-0.004992,-0.002531,-0.000704,0.001502,-0.008516,-0.008373,-0.018327,0.012343,0.001512,0.008063
4,-0.004389,-0.015599,0.009485,0.006080,0.017446,-0.005477,0.000209,-0.000308,0.005632,-0.016510,...,-0.000849,-0.005539,-0.008378,-0.004260,-0.007884,0.003775,-0.011276,-0.003164,0.023538,-0.007458


In [11]:
# calculate t-test using standardized CAAR
mean_CAAR = abnormal_returns.mean()
std_CAAR = abnormal_returns.std()
t_test_CAAR = (mean_CAAR/std_CAAR)*np.sqrt(o)
mean_SCAAR = t_test_CAAR.mean()
std_SCAAR = t_test_CAAR.std()
t_test = (mean_SCAAR/std_SCAAR)*np.sqrt(n)
t_test

-1.0731540997742544